In [1]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#os.environ['TF_ROCM_FUSION_ENABLE'] = '1'
#os.environ['MKL_DEBUG_CPU_TYPE'] = '5'

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
import gc
from sklearn.preprocessing import MinMaxScaler
import feather

In [3]:
import tensorflow as tf 
import tensorflow.keras as k
import tensorflow.keras.backend as tfb
from tcn import TCN
tf.config.set_soft_device_placement(True)
#tf.enable_eager_execution()

D:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dty

In [4]:
dateparse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y')
mpf_hsbc_raw = np.array(pd.read_csv("D:/OneDrive/MPF/HSBC SuperTrust Plus/HSBC_MPF_SuperTrustPlus.csv",parse_dates=[0], date_parser=dateparse))
daily_r, daily_c = mpf_hsbc_raw.shape
daily = np.array([s for s in mpf_hsbc_raw[:,0]]).astype("datetime64")
mpf_hsbc = mpf_hsbc_raw[:,1:].astype("float32")
mpf_hsbc_list = mpf_hsbc.transpose().tolist()
time_step = 60
batch_size = 256
lr = 0.001
minimum = 6
top = 3

mpf_hsbc_funds_name = ["AP", "B", "CA", "CE", "EE", "GB", 
                       "G", "HSIT", "HKCE", "MPFC", "NA", 
                       "S", "HSHIT", "VCAP", "VCB", "VCE", 
                       "VCUS", "GE"]
#seeds = [21251, 98689, 22528, 5447,
#         49976, 57549, 18466, 46047,
#         9292, 82205, 38882, 58473,
#         66679, 68541, 41907, 37306, 
#         997, 5609, 75506, 93963]
seeds = [21251, 98689, 5447,
         49976, 57549, 46047,
         9292, 82205, 58473,
         66679, 68541, 37306, 
         997, 5609, 93963]

In [5]:
mpf_hsbc_daily_returns = np.array([[math.log(col[i+1]/col[i]) if col[i] != 0 else -9999 for i,a in enumerate(col[1:])] for col in mpf_hsbc_list], dtype='float64').transpose()
mpf_hsbc_daily_returns[mpf_hsbc_daily_returns==-9999.0] = np.nan
mpf_hsbc_daily_returns = pd.DataFrame(mpf_hsbc_daily_returns,index=daily[1:],columns=mpf_hsbc_funds_name,dtype='float64')

mpf_hsbc_returns = mpf_hsbc_daily_returns.fillna(-999).groupby(pd.Grouper(freq='M')).sum()
mpf_hsbc_returns[mpf_hsbc_returns<-100] = np.nan

monthly_r, monthly_c = mpf_hsbc_returns.shape
monthly = mpf_hsbc_returns.index

In [6]:
#tf.config.threading.set_inter_op_parallelism_threads(0)
#tf.config.threading.set_intra_op_parallelism_threads(0)
#tf.config.set_soft_device_placement(True)


k.backend.set_floatx('float32')
k.backend.set_epsilon(1e-7)

In [7]:
mpf_hsbc_w_all = np.zeros((monthly_r, monthly_c, len(seeds)), dtype='float64')
mpf_hsbc_period = mpf_hsbc_w_all.shape[0]
max_return, min_return = mpf_hsbc_returns.max(axis=0), mpf_hsbc_returns.min(axis=0)
mpf_hsbc_returns_normalized = np.zeros((monthly_r, monthly_c), dtype='float64')
mpf_hsbc_returns_normalized = ((mpf_hsbc_returns - min_return) / (max_return - min_return)) * 2 - 1

def x_roll(x, window_size):
    na = np.zeros((len(x),window_size),dtype='float64')
    for i in range(0, len(x)):
        if len(x[0:i+1]) >= window_size:
            x_cpy = x.iloc[-window_size:].to_numpy().reshape(60)
        else:
            x_cpy = np.zeros(window_size-len(x[0:i+1]))
            x_cpy = np.append(x_cpy, x.iloc[0:i+1])
        na[i,:] = x_cpy
    return na[-60:,:]

def reset_keras():
    k.backend.clear_session()
    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

def array_generator(x,y):
    yield(x,y)
        
for pas, seed in enumerate(seeds):
    print("seed: " + str(seed))
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    k.backend.set_session(sess)
    tf.random.set_random_seed(seed)
    #tf.random.set_seed(seed)
    #tf.keras.use_session_with_seed(seed)
    
    model = k.models.Sequential([
        TCN(64, return_sequences=True),
        TCN(16, return_sequences=False),
        k.layers.Dense(1),
    ])
    adam = k.optimizers.Adam(lr, epsilon=1e-7)
    model.compile(optimizer=adam, loss='mse')
    
    #for col in (0, 1):
    for col in range(0,len(max_return)):
        print("col: " + str(col))
        counter = 0
        X = pd.DataFrame(mpf_hsbc_returns_normalized.iloc[:,col].dropna().iloc[0:], dtype='float64')
        X_month = monthly[-len(X):]
        
        y = pd.DataFrame(mpf_hsbc_returns_normalized.iloc[:,col].dropna().iloc[1:], dtype='float64')
        y_month = monthly[-len(y):]
        
        #for i in (180,181):
        for i in range(0,len(X)-1):
            print("row: " + str(i))
            if i > minimum:
                X_train = x_roll(X[0:i], time_step)
                X_train_r, X_train_t = X_train.shape
                #X_train = tf.data.Dataset.from_tensor_slices(X_train.reshape((X_train_r,X_train_t,1)).astype('float64')).batch(256)
                X_train = X_train.reshape((X_train_r,X_train_t,1)).astype('float32')
                
                X_test = (x_roll(X[0:i+1], time_step))[-1]
                #X_test = tf.data.Dataset.from_tensor_slices(X_test.reshape((1,60,1)).astype('float64')).batch(256)
                X_test = X_test.reshape((1,60,1)).astype('float32')
                
                #y_train = tf.data.Dataset.from_tensor_slices(y[0:i].to_numpy().reshape((i,1)).astype('float64')).batch(256)
                y_train = y[0:i].to_numpy().reshape((i,1)).astype('float32')[-60:]
                
                
                #print("row data prepared")
                model.fit(X_train, y_train, epochs=36, batch_size=256, use_multiprocessing=True, verbose=0)
                #print("row data fitted")
                prediction = model.predict(X_test, batch_size=256)
                #print("row data predicted")
                del [[X_train, X_train_r, X_train_t, y_train, X_test]]
            else:
                prediction = 0
            idx = (np.where(mpf_hsbc_returns.index.values == np.datetime64(X_month[i+1])))[0].tolist()
            mpf_hsbc_w_all[idx[0], col, pas] = prediction
            del prediction
            gc.collect()
        #reset_keras()
        del [[X, y, X_month, y_month]]
        gc.collect()

seed: 21251
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
col: 0
row: 0
row: 1
row: 2
row: 3
row: 4
row: 5
row: 6
row: 7
row: 8
row: 9
row: 10
row: 11
row: 12
row: 13
row: 14
row: 15
row: 16
row: 17
row: 18
row: 19
row: 20
row: 21
row: 22
row: 23
row: 24
row: 25
row: 26
row: 27
row: 28
row: 29
row: 30
row: 31
row: 32
row: 33
row: 34
row: 35
row: 36
row: 37
row: 38
row: 39
row: 40
row: 41
row: 42
row: 43
row: 44
row: 45
row: 46
row: 47
row: 48
row: 49
row: 50
row: 51
row: 52
row: 53
row: 54
row: 55
row: 56
row: 57
row: 58
row: 59
row: 60
row: 61
row: 62
row: 63
row: 64
row: 65
row: 66
row: 67
row: 68
row: 69
row: 70
row: 71
row: 72
row: 73
row: 74
row: 75
row: 76
row: 77
row: 78
row: 79
row: 80
row: 81
row: 82
row: 83
row: 84
row: 85
row: 86
row: 87
row: 88
row: 89
row: 90
row: 91
row: 92
row: 93
row: 94
row: 95
row: 96
row: 97
row: 98
row: 99
row: 100
row: 101
row: 102
row: 103
row: 104
row: 105
row: 106
row: 107
r

row: 121
row: 122
row: 123
row: 124
row: 125
row: 126
row: 127
row: 128
row: 129
row: 130
row: 131
row: 132
row: 133
row: 134
row: 135
row: 136
row: 137
row: 138
row: 139
row: 140
row: 141
row: 142
row: 143
row: 144
row: 145
row: 146
row: 147
row: 148
row: 149
row: 150
row: 151
row: 152
row: 153
row: 154
row: 155
row: 156
row: 157
row: 158
row: 159
row: 160
row: 161
row: 162
row: 163
row: 164
row: 165
row: 166
row: 167
row: 168
row: 169
row: 170
row: 171
row: 172
row: 173
row: 174
row: 175
row: 176
row: 177
row: 178
row: 179
row: 180
row: 181
row: 182
row: 183
row: 184
row: 185
row: 186
row: 187
row: 188
row: 189
row: 190
row: 191
row: 192
row: 193
row: 194
row: 195
row: 196
row: 197
row: 198
row: 199
row: 200
row: 201
row: 202
row: 203
row: 204
row: 205
row: 206
row: 207
row: 208
row: 209
row: 210
row: 211
row: 212
row: 213
row: 214
row: 215
row: 216
row: 217
row: 218
row: 219
row: 220
row: 221
row: 222
row: 223
row: 224
row: 225
row: 226
row: 227
col: 5
row: 0
row: 1
row: 2
row: 3
ro

row: 50
row: 51
row: 52
row: 53
row: 54
row: 55
row: 56
row: 57
row: 58
row: 59
row: 60
row: 61
row: 62
row: 63
row: 64
row: 65
row: 66
row: 67
row: 68
row: 69
row: 70
row: 71
row: 72
row: 73
row: 74
row: 75
row: 76
row: 77
row: 78
row: 79
row: 80
row: 81
row: 82
row: 83
row: 84
row: 85
row: 86
row: 87
row: 88
row: 89
row: 90
row: 91
row: 92
row: 93
row: 94
row: 95
row: 96
row: 97
row: 98
row: 99
row: 100
row: 101
row: 102
row: 103
row: 104
row: 105
row: 106
row: 107
row: 108
row: 109
row: 110
row: 111
row: 112
row: 113
row: 114
row: 115
row: 116
row: 117
row: 118
row: 119
row: 120
row: 121
row: 122
row: 123
row: 124
row: 125
row: 126
row: 127
row: 128
row: 129
row: 130
row: 131
row: 132
row: 133
row: 134
row: 135
row: 136
row: 137
row: 138
row: 139
row: 140
row: 141
row: 142
row: 143
row: 144
row: 145
row: 146
row: 147
row: 148
row: 149
row: 150
row: 151
row: 152
row: 153
row: 154
row: 155
row: 156
row: 157
row: 158
row: 159
row: 160
row: 161
row: 162
row: 163
row: 164
row: 165
row: 1

row: 47
row: 48
row: 49
row: 50
row: 51
row: 52
row: 53
row: 54
row: 55
row: 56
row: 57
row: 58
row: 59
row: 60
row: 61
row: 62
row: 63
row: 64
row: 65
row: 66
row: 67
row: 68
row: 69
row: 70
row: 71
row: 72
row: 73
row: 74
row: 75
row: 76
row: 77
row: 78
row: 79
row: 80
row: 81
row: 82
row: 83
row: 84
row: 85
row: 86
row: 87
row: 88
row: 89
row: 90
row: 91
row: 92
row: 93
row: 94
row: 95
row: 96
row: 97
row: 98
row: 99
row: 100
row: 101
row: 102
row: 103
col: 17
row: 0
row: 1
row: 2
row: 3
row: 4
row: 5
row: 6
row: 7
row: 8
row: 9
row: 10
row: 11
row: 12
row: 13
row: 14
row: 15
row: 16
row: 17
row: 18
row: 19
row: 20
row: 21
row: 22
row: 23
row: 24
row: 25
row: 26
row: 27
row: 28
row: 29
row: 30
row: 31
row: 32
row: 33
row: 34
row: 35
row: 36
row: 37
row: 38
row: 39
seed: 98689
col: 0
row: 0
row: 1
row: 2
row: 3
row: 4
row: 5
row: 6
row: 7
row: 8
row: 9
row: 10
row: 11
row: 12
row: 13
row: 14
row: 15
row: 16
row: 17
row: 18
row: 19
row: 20
row: 21
row: 22
row: 23
row: 24
row: 25
row: 

row: 72
row: 73
row: 74
row: 75
row: 76
row: 77
row: 78
row: 79
row: 80
row: 81
row: 82
row: 83
row: 84
row: 85
row: 86
row: 87
row: 88
row: 89
row: 90
row: 91
row: 92
row: 93
row: 94
row: 95
row: 96
row: 97
row: 98
row: 99
row: 100
row: 101
row: 102
row: 103
row: 104
row: 105
row: 106
row: 107
row: 108
row: 109
row: 110
row: 111
row: 112
row: 113
row: 114
row: 115
row: 116
row: 117
row: 118
row: 119
row: 120
row: 121
row: 122
row: 123
row: 124
row: 125
row: 126
row: 127
row: 128
row: 129
row: 130
row: 131
row: 132
row: 133
row: 134
row: 135
row: 136
row: 137
row: 138
row: 139
row: 140
row: 141
row: 142
row: 143
row: 144
row: 145
row: 146
row: 147
row: 148
row: 149
row: 150
row: 151
row: 152
row: 153
row: 154
row: 155
row: 156
row: 157
row: 158
row: 159
row: 160
row: 161
row: 162
row: 163
row: 164
row: 165
row: 166
row: 167
row: 168
row: 169
row: 170
row: 171
row: 172
row: 173
row: 174
row: 175
row: 176
row: 177
row: 178
row: 179
row: 180
row: 181
row: 182
row: 183
row: 184
row: 185
ro

...

row: 114
row: 115
row: 116
row: 117
row: 118
row: 119
row: 120
row: 121
row: 122
row: 123
row: 124
row: 125
row: 126
row: 127
row: 128
row: 129
row: 130
row: 131
row: 132
row: 133
row: 134
row: 135
row: 136
row: 137
row: 138
row: 139
row: 140
row: 141
row: 142
row: 143
row: 144
row: 145
row: 146
row: 147
row: 148
row: 149
row: 150
row: 151
row: 152
row: 153
row: 154
row: 155
row: 156
row: 157
row: 158
row: 159
row: 160
row: 161
row: 162
row: 163
row: 164
row: 165
row: 166
row: 167
row: 168
row: 169
row: 170
row: 171
row: 172
row: 173
row: 174
row: 175
row: 176
row: 177
row: 178
row: 179
row: 180
row: 181
row: 182
row: 183
row: 184
row: 185
row: 186
row: 187
row: 188
row: 189
row: 190
row: 191
row: 192
row: 193
row: 194
row: 195
row: 196
row: 197
row: 198
row: 199
row: 200
row: 201
row: 202
row: 203
row: 204
row: 205
row: 206
row: 207
row: 208
row: 209
row: 210
row: 211
row: 212
row: 213
row: 214
row: 215
row: 216
row: 217
row: 218
row: 219
row: 220
row: 221
row: 222
row: 223
row: 224
r

row: 158
row: 159
row: 160
row: 161
row: 162
row: 163
row: 164
row: 165
row: 166
row: 167
row: 168
row: 169
row: 170
row: 171
row: 172
row: 173
row: 174
row: 175
row: 176
row: 177
row: 178
row: 179
row: 180
row: 181
row: 182
row: 183
row: 184
row: 185
row: 186
row: 187
row: 188
row: 189
row: 190
row: 191
row: 192
row: 193
row: 194
row: 195
row: 196
row: 197
row: 198
row: 199
row: 200
row: 201
row: 202
row: 203
row: 204
row: 205
row: 206
row: 207
row: 208
row: 209
row: 210
row: 211
row: 212
row: 213
row: 214
row: 215
row: 216
row: 217
row: 218
row: 219
row: 220
row: 221
row: 222
row: 223
row: 224
row: 225
row: 226
row: 227
col: 11
row: 0
row: 1
row: 2
row: 3
row: 4
row: 5
row: 6
row: 7
row: 8
row: 9
row: 10
row: 11
row: 12
row: 13
row: 14
row: 15
row: 16
row: 17
row: 18
row: 19
row: 20
row: 21
row: 22
row: 23
row: 24
row: 25
row: 26
row: 27
row: 28
row: 29
row: 30
row: 31
row: 32
row: 33
row: 34
row: 35
row: 36
row: 37
row: 38
row: 39
row: 40
row: 41
row: 42
row: 43
row: 44
row: 45
row:

row: 17
row: 18
row: 19
row: 20
row: 21
row: 22
row: 23
row: 24
row: 25
row: 26
row: 27
row: 28
row: 29
row: 30
row: 31
row: 32
row: 33
row: 34
row: 35
row: 36
row: 37
row: 38
row: 39
row: 40
row: 41
row: 42
row: 43
row: 44
row: 45
row: 46
row: 47
row: 48
row: 49
row: 50
row: 51
row: 52
row: 53
row: 54
row: 55
row: 56
row: 57
row: 58
row: 59
row: 60
row: 61
row: 62
row: 63
row: 64
row: 65
row: 66
row: 67
row: 68
row: 69
row: 70
row: 71
row: 72
row: 73
row: 74
row: 75
row: 76
row: 77
row: 78
row: 79
row: 80
row: 81
row: 82
row: 83
row: 84
row: 85
row: 86
row: 87
row: 88
row: 89
row: 90
row: 91
row: 92
row: 93
row: 94
row: 95
row: 96
row: 97
row: 98
row: 99
row: 100
row: 101
row: 102
row: 103
row: 104
row: 105
row: 106
row: 107
row: 108
row: 109
row: 110
row: 111
row: 112
row: 113
row: 114
row: 115
row: 116
row: 117
row: 118
row: 119
row: 120
row: 121
row: 122
row: 123
row: 124
row: 125
row: 126
row: 127
row: 128
row: 129
row: 130
row: 131
row: 132
row: 133
row: 134
row: 135
row: 136
row

row: 64
row: 65
row: 66
row: 67
row: 68
row: 69
row: 70
row: 71
row: 72
row: 73
row: 74
row: 75
row: 76
row: 77
row: 78
row: 79
row: 80
row: 81
row: 82
row: 83
row: 84
row: 85
row: 86
row: 87
row: 88
row: 89
row: 90
row: 91
row: 92
row: 93
row: 94
row: 95
row: 96
row: 97
row: 98
row: 99
row: 100
row: 101
row: 102
row: 103
row: 104
row: 105
row: 106
row: 107
row: 108
row: 109
row: 110
row: 111
row: 112
row: 113
row: 114
row: 115
row: 116
row: 117
row: 118
row: 119
row: 120
row: 121
row: 122
row: 123
row: 124
row: 125
row: 126
row: 127
row: 128
row: 129
row: 130
row: 131
row: 132
row: 133
row: 134
row: 135
row: 136
row: 137
row: 138
row: 139
row: 140
row: 141
row: 142
row: 143
row: 144
row: 145
row: 146
row: 147
row: 148
row: 149
row: 150
row: 151
row: 152
row: 153
row: 154
row: 155
row: 156
row: 157
row: 158
row: 159
row: 160
row: 161
row: 162
row: 163
row: 164
row: 165
row: 166
row: 167
row: 168
row: 169
row: 170
row: 171
row: 172
row: 173
row: 174
row: 175
row: 176
row: 177
row: 178
r

row: 112
row: 113
row: 114
row: 115
row: 116
row: 117
row: 118
row: 119
row: 120
row: 121
row: 122
row: 123
row: 124
row: 125
row: 126
row: 127
row: 128
row: 129
row: 130
row: 131
row: 132
row: 133
row: 134
row: 135
row: 136
row: 137
row: 138
row: 139
row: 140
row: 141
row: 142
row: 143
row: 144
row: 145
row: 146
row: 147
row: 148
row: 149
row: 150
row: 151
row: 152
row: 153
row: 154
row: 155
row: 156
row: 157
row: 158
row: 159
row: 160
row: 161
row: 162
row: 163
row: 164
row: 165
row: 166
row: 167
row: 168
row: 169
row: 170
row: 171
row: 172
row: 173
row: 174
row: 175
row: 176
row: 177
row: 178
row: 179
row: 180
row: 181
row: 182
row: 183
row: 184
row: 185
row: 186
row: 187
row: 188
row: 189
row: 190
row: 191
row: 192
row: 193
row: 194
row: 195
row: 196
row: 197
row: 198
row: 199
row: 200
row: 201
row: 202
row: 203
row: 204
row: 205
row: 206
row: 207
row: 208
row: 209
row: 210
row: 211
row: 212
row: 213
row: 214
row: 215
row: 216
row: 217
row: 218
row: 219
row: 220
row: 221
row: 222
r

In [9]:
for i in range(1,16):
    feather.write_dataframe(pd.DataFrame(mpf_hsbc_w_all[:,:,i-1]), "D:/GitHub repo/MPF-ANN/temp/mpf_hsbc_tcn_w_all.feather"+str(i))